In [63]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from joblib import load

In [87]:
typhi_grna = "./gRNA_sequences/typhi_grna.csv"
typhi_records = pd.read_csv(typhi_grna)
typhi_records['gRNA'] = typhi_records['gRNA'].str.lower()
typhi_records.gRNA[0],typhi_records.Type[0]


('aaagataccagagcccgaat', 'sty1607')

In [84]:
clf = load('Models/grnamodel_ds.joblib')
output_ds = pd.read_pickle("./Outputs/df_feat_ds_doubletree.pkl") 
output_ds = output_ds[output_ds.importance_values>0.01]
output_ds

,features,importance_values
690,Motif: ta,0.079278
625,gc_target,0.040530
258,"targetposition41, g",0.024618
647,Motif: c,0.022447
685,Motif: gta,0.019166
634,Motif: acc,0.018614
627,Motif: aa,0.018292
668,Motif: g,0.023565
653,Motif: cc,0.016772
629,Motif: aac,0.015493


In [85]:
def one_hot_encode(seq):
    mapping = dict(zip("acgt", range(4)))    
    seq2 = [mapping[i] for i in seq]
    return np.eye(4)[seq2]
def one_hot_encode_duplets(seq):
    mapping = dict(zip(('aa', 'ac', 'ag', 'at', 'ca', 'cc', 'cg', 'ct', 'ga', 'gc', 'gg', 'gt', 'ta', 'tc', 'tg', 'tt'), range(16)))
    seq2 = []
    for i in range(len(seq)-1):
      seq2.append(mapping[seq[i:i+2]])
    return np.eye(16)[seq2]
def calcGC(seq):
  gc_content = [] 
  count = 0
  for l in range(len(seq)):
    if seq[l] == 'g' or seq[l] == 'c':
      count += 1
  return count/len(seq)
def countMotifs(seq, motifs):
  counts = []
  for motif in motifs:
    count = 0
    for i in range(len(seq) - len(motif)):
      if seq[i:i+len(motif)] == motif:
        count += 1
    counts.append(count/(len(seq) - len(motif)))
  return counts
bases = ['a','c','g','t']
motifs_to_look_for = []
for i in range(len(bases)):
  motifs_to_look_for.append(bases[i])
  for j in range(len(bases)):
    motifs_to_look_for.append(bases[i]+bases[j])
    for k in range(len(bases)):
      motifs_to_look_for.append(bases[i]+bases[j]+bases[k])
      # for l in range(len(bases)):
      #   motifs_to_look_for.append(bases[i]+bases[j]+bases[k]+bases[l])
motifs_to_look_for.append('aaaa')
motifs_to_look_for.append('cccc')
motifs_to_look_for.append('gggg')
motifs_to_look_for.append('tttt')

In [89]:
X_ds = []
true_typhi_records = []
true_typhi_type = []
k = 20
for i in range(len(typhi_records)):
  ## 1 and 33 are not length 20
  if len(typhi_records.gRNA[i])!=20:
    continue
  else: 
    true_typhi_records.append(typhi_records.gRNA[i])
    true_typhi_type.append(typhi_records.Type[i])
  t = np.concatenate((one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(typhi_records.gRNA[i]), one_hot_encode(typhi_records.gRNA[i]), one_hot_encode_duplets(typhi_records.gRNA[i])
                           , calcGC(typhi_records.gRNA[i]), calcGC(typhi_records.gRNA[i]), countMotifs(typhi_records.gRNA[i], motifs_to_look_for)), axis = None)
  X_ds.append(t)
X_reduced_ds = np.array(X_ds)[:,output_ds.index]

In [90]:
y_pred = clf.predict(X_reduced_ds)
y_pred

array([1.23276282, 1.21147474, 1.29585314, 1.60880267, 1.60090593,
       1.08381216, 1.2227505 , 0.93718764, 1.30818427, 0.97702069,
       1.23182215, 1.31992997, 0.94517124, 1.45697363, 1.33013818,
       1.44376645, 1.5750024 , 1.22009383, 1.43346411, 1.57324821,
       1.47826796, 1.00909247, 1.67010306, 1.50052008, 1.45729205,
       1.0663571 , 1.02370029, 1.9355428 , 1.01484841, 1.36995786,
       1.41226364, 1.7693148 , 1.34159118, 1.1205813 , 1.64203889,
       1.58406682, 1.28982707, 1.29649926, 1.35334844, 1.23894574,
       0.96813284, 1.2598804 , 0.95271202, 2.03398478, 1.41559137,
       1.38388034, 1.65410357, 0.81244153, 0.71160343, 1.18908884,
       1.74726626, 0.79485502, 1.13025061, 1.3764529 , 1.43959012])

In [94]:
df = pd.DataFrame({'gRNA':true_typhi_records, 'type':true_typhi_type, 'predicted_ds_k':y_pred})
df = df.sort_values('predicted_ds_k', ascending = False)

df.to_csv('./Outputs/typhi_kvalues_prediction.csv')